# Client 2 Gas

### Imports

In [1]:
import sys
import warnings
import os.path
import statistics
import math
sys.path.insert(0, os.path.abspath('../../../../functions'))
from knmi_weather import knmi_weather as knmi # eda tools
from sp_ql import Engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
from math import ceil
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RepeatedKFold
import joblib
import seaborn as sns
import xgboost
from xgboost import XGBRegressor

### Defs

In [2]:
def calc_soc_pos_on_plot(df_compare_monthly, soc):
    sm = list(df_compare_monthly.index < soc).index(False) # first False is start month
    return sm - 0.5 # place of dashed line on plot in 0-1 x coord range

### Read Inputs

In [ ]:
project = 'c2'
engine=Engine(project=project, isAsync=False)
bms_registers = engine.list().filter(unit='CELCIUS').all()
build_names = bms_registers['keyname_01'].unique()
build_names = build_names[~pd.isna(build_names)]
eans=pd.read_csv(f'/Users/Matt/Desktop/MSc/Thesis/Spectral/brighter_eda/inputs/eans/{project.lower()}_eans.csv')
gas_eans=eans[eans['Contract Type']=='Natural Gas']
gas_eans=gas_eans[['keyname_01', 'EAN']].dropna().set_index('keyname_01').squeeze()

adds missing buildings
for building in build_names:
    if building not in gas_eans.index:
        gas_eans[building]=np.nan
#nodata
del gas_eans['V******']
del gas_eans['A******']
del gas_eans['S******']
del gas_eans['H******']
del gas_eans['TR*****']
del gas_eans['TU*****']
del gas_eans['KR*****']
#missing or gappy data
del gas_eans['HG*****']
del gas_eans['OV*****']
socs = pd.read_csv(r'/Users/Matt/Desktop/MSc/Thesis/Spectral/brighter_eda/inputs/start_of_control/start_of_control_dates.csv')
socs = socs[socs['project']==project]
socs = socs.set_index('keyname_01')['soc_date'].squeeze()
socs = pd.to_datetime(socs)

In [64]:
build_names=np.delete(build_names, 10, axis=None) #no gas or electr data

In [65]:
# determine building location
cities = eans[['keyname_01', 'City']].dropna()
cities = cities.drop_duplicates().set_index('keyname_01').squeeze()
cities['P*****']='Amsterdam' #manually adding: *****
cities = cities[gas_eans.index]

# read and preprocess weather data
knmi_station_lookup = pd.read_csv(r'../../../../inputs/knmi/knmi_uurgegevens/city_knmi_station_lookup.csv', index_col=0, squeeze=True)

In [66]:
weather = {}
for station in knmi_station_lookup.unique():
    knmi_weather_file_1 = f'../../../../inputs/knmi/knmi_uurgegevens/uurgeg_{station}_2011-2020.txt'
    knmi_weather_file_2 = f'../../../../inputs/knmi/knmi_uurgegevens/uurgeg_{station}_2021-2030.txt'
    knmi_weather = pd.read_csv(knmi_weather_file_1,
                               skiprows=31,
                               skipinitialspace=True,
                               usecols=list(range(1, 25)))
    knmi_weather = knmi_weather.append(pd.read_csv(knmi_weather_file_2,
                                                   skiprows=31,
                                                   skipinitialspace=True,
                                                   usecols=list(range(1, 25))))
    # making proper index for KNMI weather data
    knmi_weather.index = knmi.convert_knmi_dates(date=knmi_weather['YYYYMMDD'],
                                                 hour=knmi_weather['HH'])
    # clean up values
    knmi_weather = knmi.cleanup_knmi_units(knmi_weather=knmi_weather)
    weather[station] = knmi_weather

In [67]:
## Determine training and prediction intervals
train_periods = pd.DataFrame()
train_periods['end'] = socs[gas_eans.index]
train_periods['start'] = socs[gas_eans.index] - dt.timedelta(days=365) #more training data=better

In [68]:
## prepare features and training data:
# get and save training gas use data locally (no need to do this every time)
engine = Engine(project=project, isAsync=False)
time_res = 'd'
gas_use_train = pd.DataFrame(index=pd.date_range(start=train_periods['start'].min(),
                                                 end=train_periods['end'].max(),
                                                 freq=time_res))

# data from sp-ql
for building in gas_eans.index:
    # get gas use
    start_train = train_periods.loc[building, 'start']
    end_train = train_periods.loc[building, 'end']
    gas_ean = gas_eans[building]
    gas_use = engine.select(gas_ean).group_by(time_res).sum().after(start_train).before(end_train).all()
    gas_use = gas_use.rename(columns={gas_use.columns[0]:building})
    gas_use_train = gas_use_train.join(gas_use)
gas_use_train.to_csv(r'c2_gas/data/gas_use_train.csv')

In [69]:
gas_use_train = pd.read_csv(r'c2_gas/data/gas_use_train.csv', index_col=0,
                            parse_dates=True)

In [ ]:
# some buildings have small gaps in gas data. We can interpolate over these
for building in gas_eans.index:
    start_train = train_periods.loc[building, 'start']
    end_train = train_periods.loc[building, 'end']
    gas_use_train_build = gas_use_train.loc[start_train:end_train, building]
    # setting first element to 0 if nan. To aid interpolation.
    if gas_use_train_build.iloc[[0]].isna()[0]:
        gas_use_train_build.iloc[0] = 0
    gas_use_train_build = gas_use_train_build.interpolate(limit=10) # filling max 10 consecutive days of missing data
    gas_use_train.loc[start_train:end_train, building] = gas_use_train_build

### RF model

In [ ]:
for building in gas_eans.index:
    print(fr'Training building {building}, {gas_eans.index.get_loc(building) + 1}/{len(gas_eans)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = train_periods.loc[building, 'start']
    end_train = train_periods.loc[building, 'end']
    #
    features = pd.DataFrame()
    features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
    features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
    features['dow'] = features.index.dayofweek
    #features['month'] = features.index.month
    #
    gas_use_train_build = gas_use_train[building]
    # X: training features, y: training target
    X = features[(features.index >= start_train) & (features.index < end_train)]
    y = gas_use_train_build[(gas_use_train_build.index >= start_train) &
                            (gas_use_train_build.index < end_train)]
    #
    # checking if there is any nan in the data
    if X.isna().sum().sum() + y.isna().sum() > 0:
        raise ValueError('NaN in training data:', building)
    #
    # checking on dependency of taget on features to get a feel
    if False:
        fig, ax = plt.subplots(len(X.columns), 1)
        for i, c in enumerate(X.columns):
            ax[i].scatter(x=X[c], y=y)
            ax[i].set_title(c)
        plt.show(block=False)
    #
    # make train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    #
    # grid-search for hyper parameter tuning
    param_grid = {'max_depth': [2, 5, 10, 20, 50, 100],
                  'min_samples_leaf': [1, 5, 10],
                  'n_estimators': [500]}
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                               cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)
    #
    # save the trained RF
    joblib.dump(grid_search.best_estimator_, fr'c2_gas/data/trained_models/rf/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_gas/data/trained_models/rf/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_gas/data/trained_models/rf/y_test_{building}.joblib')

### Gradient Boosting

In [ ]:
for building in gas_eans.index:
    print(fr'Training building {building}, {gas_eans.index.get_loc(building) + 1}/{len(gas_eans)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = train_periods.loc[building, 'start']
    end_train = train_periods.loc[building, 'end']
    #
    features = pd.DataFrame()
    features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
    features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
    features['dow'] = features.index.dayofweek
    #features['month'] = features.index.month
    #
    gas_use_train_build = gas_use_train[building]
    # X: training features, y: training target
    X = features[(features.index >= start_train) & (features.index < end_train)]
    y = gas_use_train_build[(gas_use_train_build.index >= start_train) &
                            (gas_use_train_build.index < end_train)]
    #
    # checking if there is any nan in the data
    if X.isna().sum().sum() + y.isna().sum() > 0:
        raise ValueError('NaN in training data:', building)
    #
    # checking on dependency of taget on features to get a feel
    if False:
        fig, ax = plt.subplots(len(X.columns), 1)
        for i, c in enumerate(X.columns):
            ax[i].scatter(x=X[c], y=y)
            ax[i].set_title(c)
        plt.show(block=False)
    #
    # make train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    #
    # grid-search for hyper parameter tuning
    param_grid = {
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "max_depth":[3,5,8],
    "max_features":["log2", "sqrt"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10],
    }
    
    rf = GradientBoostingRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                               cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)
    
    #
    # save the trained RF
    joblib.dump(grid_search.best_estimator_, fr'c2_gas/data/trained_models/gb/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_gas/data/trained_models/gb/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_gas/data/trained_models/gb/y_test_{building}.joblib')

### XGBoosting

In [ ]:
for building in gas_eans.index:
    print(fr'Training building {building}, {gas_eans.index.get_loc(building) + 1}/{len(gas_eans)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = train_periods.loc[building, 'start']
    end_train = train_periods.loc[building, 'end']
    features = pd.DataFrame()
    
    features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
    features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
    features['dow'] = features.index.dayofweek

    gas_use_train_build = gas_use_train[building]
    X = features[(features.index >= start_train) & (features.index < end_train)]
    y = gas_use_train_build[(gas_use_train_build.index >= start_train) &
                            (gas_use_train_build.index < end_train)]
    if X.isna().mean().mean() + y.isna().mean() > 0:
        raise ValueError('NaN in training data:', building)
    if False:
        fig, ax = plt.subplots(len(X.columns), 1)
        for i, c in enumerate(X.columns):
            ax[i].scatter(x=X[c], y=y)
            ax[i].set_title(c)
        plt.show(block=False)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    
    param_grid={
                "learning_rate": (0.05, 0.10, 0.15, 0.2),
                "max_depth": [3, 4, 5, 6, 8],
                "min_child_weight": [1, 3, 5, 7]
                }

    rf = XGBRegressor()
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                               cv=cv, n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)

    joblib.dump(grid_search.best_estimator_, fr'c2_gas/data/trained_models/xg/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_gas/data/trained_models/xg/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_gas/data/trained_models/xg/y_test_{building}.joblib')

### Metrics

In [72]:
# reading in saved trained RF models and calculating mae for the test set
mae_range_percent = pd.Series()
for building in gas_eans.index:
    models = os.listdir('c2_gas/data/trained_models/rf') #change for RF, GB, XG
    if fr'trained_{building}.joblib' in models:
        grid_search = joblib.load(fr'c2_gas/data/trained_models/rf/trained_{building}.joblib') #change for RF, GB, XG
        X_test = joblib.load(fr'c2_gas/data/trained_models/rf/X_test_{building}.joblib') #change for RF, GB, XG
        y_test = joblib.load(fr'c2_gas/data/trained_models/rf/y_test_{building}.joblib') #change for RF, GB, XG
    #
    # grid search already contains the re-trained model with the best parameters
    y_test_pred = grid_search.predict(X_test)
    #
    # calculating MAE and plotting test and pred_test
    mae = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
    # mae in percent of range
    mae_range_percent[building] = mae / y_test.max() * 100

In [73]:
statistics.mean(mae_range_percent)

5.460624247899692

### Metrics together - MAE

In [74]:
metric_com_mae = pd.DataFrame(index=gas_eans.index)
regressors = ['rf', 'gb', 'xg']
for regressor in regressors:
    mae_range_percent = pd.Series()
    for building in gas_eans.index:
        models = os.listdir(f'c2_gas/data/trained_models/{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_gas/data/trained_models/{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_gas/data/trained_models/{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
        #
        # grid search already contains the re-trained model with the best parameters
        y_test_pred = grid_search.predict(X_test)

        #
        # calculating MAE and plotting test and pred_test
        mae = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
        # mae in percent of range
        mae_range_percent[building] = mae / y_test.max() * 100

        gas_use_train_build = gas_eans[building]

    metric_com_mae[regressor.upper()]=mae_range_percent

### Metrics together - RMSE

In [75]:
metric_com_rmse = pd.DataFrame(index=gas_eans.index)
regressors = ['rf', 'gb', 'xg']
for regressor in regressors:
    rmse_range_percent = pd.Series()
    for building in gas_eans.index:
        models = os.listdir(f'c2_gas/data/trained_models/{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_gas/data/trained_models/{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_gas/data/trained_models/{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
        #
        # grid search already contains the re-trained model with the best parameters
        y_test_pred = grid_search.predict(X_test)
        
        #calculating RMSE
        MSE = np.square(np.subtract(y_test, y_test_pred)).mean() 
        RMSE = math.sqrt(MSE)
        rmse_range_percent[building]=RMSE / y_test.max() * 100

    metric_com_rmse[regressor.upper()]=rmse_range_percent

### Stacked ensembling - MAE

In [ ]:
en_mae = pd.DataFrame(index=gas_eans.index)
regressors = ['rf', 'gb', 'xg']
mae_range_percent = pd.Series()
master_test=pd.DataFrame()
master_test_pred=pd.DataFrame()

df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df_c=pd.DataFrame()

df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df_d=pd.DataFrame()

for building in gas_eans.index:
    for regressor in regressors:
        models = os.listdir(f'c2_gas/data/trained_models/{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_gas/data/trained_models/{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_gas/data/trained_models/{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
            #print(y_test) ##gives diffent dates, so needs to be investigated --> tried setting random seed during training
            y_test_pred = grid_search.predict(X_test)
            
            if regressor=="rf":
                df1=y_test.to_frame()
                df4=pd.DataFrame(y_test_pred, columns=[building])
                df1.reset_index(drop=True, inplace=True)
                df4.reset_index(drop=True, inplace=True)
            if regressor=="gb":
                df2=y_test.to_frame()
                df5=pd.DataFrame(y_test_pred, columns=[building])
                df2.reset_index(drop=True, inplace=True)
                df5.reset_index(drop=True, inplace=True)
            if regressor=="xg":
                df3=y_test.to_frame()
                df6=pd.DataFrame(y_test_pred, columns=[building])
                df3.reset_index(drop=True, inplace=True)
                df6.reset_index(drop=True, inplace=True)
                
    df_c=pd.concat([df1, df2, df3], axis=1)
    df_c.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_c[building+"_en"]=(df_c[building+"_rf"]+df_c[building+"_gb"]+df_c[building+"_xg"])/3
    del df_c[building+"_rf"]
    del df_c[building+"_gb"]
    del df_c[building+"_xg"]
    
    df_d=pd.concat([df4, df5, df6], axis=1)
    df_d.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_d[building+"_en"]=(df_d[building+"_rf"]+df_d[building+"_gb"]+df_d[building+"_xg"])/3
    
    del df_d[building+"_rf"]
    del df_d[building+"_gb"]
    del df_d[building+"_xg"]

    mae = mean_absolute_error(y_true=df_c[building+"_en"], y_pred=df_d[building+"_en"])
    
    mae_range_percent[building] = mae / df_c[building+"_en"].max() * 100

    en_mae['EN']=mae_range_percent
    

### Ensemble stacked - MAE

In [ ]:
metric_com_mae['EN']=en_mae
print(metric_com_mae)
print("\n")
print("Average MAEs")
round(metric_com_mae.mean(), 5)

### Stacked ensembling - RMSE

In [78]:
en_rmse = pd.DataFrame(index=gas_eans.index)
regressors = ['rf', 'gb', 'xg']
rmse_range_percent = pd.Series()
master_test=pd.DataFrame()
master_test_pred=pd.DataFrame()

df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df_c=pd.DataFrame()

df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df_d=pd.DataFrame()

for building in gas_eans.index:
    for regressor in regressors:
        models = os.listdir(f'c2_gas/data/trained_models/{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_gas/data/trained_models/{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_gas/data/trained_models/{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
            #print(y_test) ##gives diffent dates, so needs to be investigated --> tried setting random seed during training
            y_test_pred = grid_search.predict(X_test)
            
            if regressor=="rf":
                df1=y_test.to_frame()
                df4=pd.DataFrame(y_test_pred, columns=[building])
                df1.reset_index(drop=True, inplace=True)
                df4.reset_index(drop=True, inplace=True)
            if regressor=="gb":
                df2=y_test.to_frame()
                df5=pd.DataFrame(y_test_pred, columns=[building])
                df2.reset_index(drop=True, inplace=True)
                df5.reset_index(drop=True, inplace=True)
            if regressor=="xg":
                df3=y_test.to_frame()
                df6=pd.DataFrame(y_test_pred, columns=[building])
                df3.reset_index(drop=True, inplace=True)
                df6.reset_index(drop=True, inplace=True)
                
    df_c=pd.concat([df1, df2, df3], axis=1)
    df_c.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_c[building+"_en"]=(df_c[building+"_rf"]+df_c[building+"_gb"]+df_c[building+"_xg"])/3
    del df_c[building+"_rf"]
    del df_c[building+"_gb"]
    del df_c[building+"_xg"]
    
    df_d=pd.concat([df4, df5, df6], axis=1)
    df_d.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_d[building+"_en"]=(df_d[building+"_rf"]+df_d[building+"_gb"]+df_d[building+"_xg"])/3
    

    MSE = np.square(np.subtract(df_c[building+"_en"], df_d[building+"_en"])).mean() 
    RMSE = math.sqrt(MSE)
    rmse_range_percent[building]=RMSE / df_c[building+"_en"].max() * 100
    
    en_rmse['EN']=rmse_range_percent
    

### Ensemble effort - RMSE

In [ ]:
metric_com_rmse['EN']=en_rmse
print(metric_com_rmse)
print("\n")
print("Average RMSEs")
round(metric_com_rmse.mean(), 5)

# All models

### Adding ensemble predictions

In [81]:
gas_use_pred = pd.DataFrame(index=features.index)
for m in ['rf','gb','xg']:
    for building in gas_eans.index:
        # select weather data for prediction
        city = cities[building]
        knmi_station = knmi_station_lookup[city]
        knmi_weather = weather[knmi_station]
        # preapare features for prediction
        features = pd.DataFrame()
        features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
        features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
        features['dow'] = features.index.dayofweek
        #features['month'] = features.index.month
        X_pred = features
        # do the predictions
        models = os.listdir(f'c2_gas/data/trained_models/{m}')
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/{m}/trained_{building}.joblib')
        y_pred = grid_search.predict(X_pred)
        gas_use_pred[building+'_'+m] = y_pred

In [82]:
#ensemble stacking, equal weight
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3

In [83]:
# compare predicted and measured gas use after start of control
engine = Engine(project=project, isAsync=False)
time_res = 'd'
gas_use_meas = pd.DataFrame(index=pd.date_range(start=train_periods['start'].min(),
                                                 end=knmi_weather.index[-1],
                                                 freq=time_res))
for building in gas_eans.index:
    # get gas use
    start = train_periods['start'].min()
    end = knmi_weather.index[-1]
    gas_ean = gas_eans[building]
    gas_use = engine.select(gas_ean).group_by(time_res).sum().after(start).before(end).all()
    gas_use = gas_use.rename(columns={gas_use.columns[0]:building})
    gas_use_meas = gas_use_meas.join(gas_use)

In [85]:
gas_use_meas.rename(columns = {'*****':'*****_rf', '*****':'*****_rf', '*****':'*****_rf', 
                                     '*****':'*****_rf', '*****':'*****_rf', '*****':'*****_rf',
                                    '*****':'*****_rf', '*****':'*****_rf'}, inplace = True)

In [86]:
for m in ['gb','xg']:
    for building in gas_eans.index:
        gas_use_meas[building+"_"+m]=gas_use_meas[building+"_rf"]

In [87]:
month_savings = {}
month_savings_m3 = {}
total_savings_perc = pd.Series()
total_savings_m3 = pd.Series()

for m in ['rf','gb','xg']:
    for building in gas_use_meas.columns:
        size = len(building)
        start = train_periods.loc[building[:size-3], 'start']
        end = knmi_weather.index[-1]
        soc = socs[building[:size-3]]
        gas_use_meas_m = gas_use_meas.loc[start:end, building].resample('d').sum()
        gas_use_pred_m = gas_use_pred.loc[start:end, building].resample('d').sum()
        month_saving = (gas_use_pred_m - gas_use_meas_m) / gas_use_pred_m * 100
        month_savings[building] = month_saving
        month_savings_m3[building] = gas_use_pred_m - gas_use_meas_m
        total_savings_m3[building] = (gas_use_pred.loc[soc:, building] - gas_use_meas.loc[soc:, building]).sum()
        total_savings_perc[building] = total_savings_m3[building] / gas_use_pred.loc[soc:, building].sum() * 100

train_pred_mae = pd.Series()
for m in ['rf','gb','xg']:
    for building in gas_use_meas.columns:
        size = len(building)
        soc = socs[building[:size-3]]
        start = train_periods.loc[building[:size-3], 'start']
        end = train_periods.loc[building[:size-3], 'end'] #knmi_weather.index[-1]
        gas_use_meas_m = gas_use_meas.loc[start:end, building].resample('d').sum()
        gas_use_pred_m = gas_use_pred.loc[start:end, building].resample('d').sum()
        gas_use_month_mae = mean_absolute_error(y_true=gas_use_meas_m, y_pred=gas_use_pred_m)
        train_pred_mae[building] = gas_use_month_mae

In [ ]:
for building in gas_eans.index:
    t=pd.DataFrame()
    fig, ax = plt.subplots(figsize=(25, 8))
    gas_use_pred[building+"_rf"].plot(ax=ax, color='green')
    gas_use_pred[building+"_gb"].plot(ax=ax, color='blue')
    gas_use_pred[building+"_xg"].plot(ax=ax, color='red')
    t[building+"_measured"]=gas_use_meas[building+"_rf"]
    t.plot(ax=ax, color='gold')
    #electr_use_pred_en[building].plot(ax=ax, color='black')
    ax.set_xlim(left=gas_use_train[building].dropna().index[0])
    ax.set_title(building)
    ax.set_ylabel('Electricity use (kWh)')
    ax.legend()
    ax.axvline(x=socs[building], color='k', ls='--')
    plt.show(block=False)

### Comparison of individual models

In [132]:
gas_use_pred = pd.DataFrame(index=features.index)
for building in gas_eans.index:
    # select weather data for prediction
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    # preapare features for prediction
    features = pd.DataFrame()
    features['daily_mean_OAT'] = knmi_weather['T'].resample('d').mean()
    features['daily_sum_GHI'] = knmi_weather['Q'].resample('d').sum()
    features['dow'] = features.index.dayofweek
    #features['month'] = features.index.month
    X_pred = features
    # do the predictions
    models = os.listdir('c2_gas/data/trained_models/xg') #change for RF, GB, XG
    if fr'trained_{building}.joblib' in models:
        grid_search = joblib.load(fr'c2_gas/data/trained_models/xg/trained_{building}.joblib') #change for RF, GB, XG
    y_pred = grid_search.predict(X_pred)
    gas_use_pred[building] = y_pred

In [133]:
# compare predicted and measured gas use after start of control
engine = Engine(project=project, isAsync=False)
time_res = 'd'
gas_use_meas = pd.DataFrame(index=pd.date_range(start=train_periods['start'].min(),
                                                 end=knmi_weather.index[-1],
                                                 freq=time_res))
for building in gas_eans.index:
    # get gas use
    start = train_periods['start'].min()
    end = knmi_weather.index[-1]
    gas_ean = gas_eans[building]
    gas_use = engine.select(gas_ean).group_by(time_res).sum().after(start).before(end).all()
    gas_use = gas_use.rename(columns={gas_use.columns[0]:building})
    gas_use_meas = gas_use_meas.join(gas_use)

In [134]:
# calculate savings
# saving = (pred - meas)/pred
# storing in a dict, because each building has different number of months
month_savings = {}
month_savings_m3 = {}
total_savings_perc = pd.Series()
total_savings_m3 = pd.Series()
for building in gas_eans.index:
    #building = 'GPR_EH'
    start = train_periods.loc[building, 'start']
    end = knmi_weather.index[-1]
    soc = socs[building]
    # monthly savings (including the training preiod where savings should be ~0)
    gas_use_meas_m = gas_use_meas.loc[start:end, building].resample('m').sum()
    gas_use_pred_m = gas_use_pred.loc[start:end, building].resample('m').sum()
    month_saving = (gas_use_pred_m - gas_use_meas_m) / gas_use_pred_m * 100
    month_savings[building] = month_saving
    month_savings_m3[building] = gas_use_pred_m - gas_use_meas_m
    # total savings after SOC
    total_savings_m3[building] = (gas_use_pred.loc[soc:, building] - gas_use_meas.loc[soc:, building]).sum()
    total_savings_perc[building] = total_savings_m3[building] / gas_use_pred.loc[soc:, building].sum() * 100

# calculate prediction error for error bars
train_pred_mae = pd.Series()
for building in gas_eans.index:
    soc = socs[building]
    start = train_periods.loc[building, 'start']
    end = train_periods.loc[building, 'end'] #knmi_weather.index[-1]
    gas_use_meas_m = gas_use_meas.loc[start:end, building].resample('m').sum()
    gas_use_pred_m = gas_use_pred.loc[start:end, building].resample('m').sum()
    #soc_pos = calc_soc_pos_on_plot(df_compare_monthly=gas_use_meas_m, soc=soc)
    #gas_use_meas_m = gas_use_meas_m.iloc[:int(soc_pos)]
    #gas_use_pred_m = gas_use_pred_m.iloc[:int(soc_pos)]
    gas_use_month_mae = mean_absolute_error(y_true=gas_use_meas_m, y_pred=gas_use_pred_m)
    #train_pred_error = (gas_use_pred_m - gas_use_meas_m) / gas_use_pred_m * 100
    train_pred_mae[building] = gas_use_month_mae

In [ ]:
for building in gas_eans.index:
    total_savings_m3[building] = (gas_use_pred.loc[soc:, building] - gas_use_meas.loc[soc:, building]).sum()
    total_savings_perc[building] = ((gas_use_pred.loc[soc:, building] - gas_use_meas.loc[soc:, building]).sum() / gas_use_pred.loc[soc:, building].sum()) * 100
total_savings_perc

In [ ]:
for building in gas_eans.index:
    start = train_periods.loc[building, 'start']
    end = knmi_weather.index[-1]
    df_compare = pd.DataFrame(index=pd.date_range(start=start,
                                                     end=end,
                                                     freq=time_res))
    df_compare = df_compare.join(gas_use_meas[[building]])
    df_compare = df_compare.rename(columns={building:fr'{building}_meas'})
    df_compare = df_compare.join(gas_use_pred[[building]])
    df_compare = df_compare.rename(columns={building:fr'{building}_pred'})
    #
    df_compare = df_compare.resample('m').sum()
    soc = socs[building]#.date()
    soc_pos = calc_soc_pos_on_plot(df_compare_monthly=df_compare, soc=soc)
    train_end_pos = calc_soc_pos_on_plot(df_compare_monthly=df_compare, soc=train_periods.loc[building, 'end'])
    #
    fig, ax = plt.subplots(figsize=(25, 8))
    err = pd.DataFrame(index=df_compare.index, data={fr'{building}_pred':train_pred_mae[building]})
    err.iloc[:int(soc_pos) + 1] = np.nan
    df_compare.plot.bar(ax=ax, color=['gold', 'darkgray'], yerr=err,
                        error_kw=dict(ecolor='darkred', lw=1, capsize=2, capthick=1))
    #soc_mark = pd.Series(index=df_compare.index, data=np.nan)
    ax.set_xticklabels([x.strftime("%Y-%m") for x in df_compare.index], rotation=45)
    plt.vlines(x=train_end_pos, ymin=0, ymax=df_compare.max().max(), color='darkred', ls='--')
    plt.vlines(x=soc_pos, ymin=0, ymax=df_compare.max().max(), color='k', ls='--')
    # adding saving % text
    patches = ax.patches
    patches_m = patches[0:int(len(patches)/2)]
    patches_p = patches[int(len(patches)/2):]
    for i, p in enumerate(patches[0:int(len(patches)/2)]):
        if i > soc_pos:
            saving = int(month_savings[building].iloc[i].round(0))
            saving_m3 = int(month_savings_m3[building].iloc[i].round(0))
            width = p.get_width()
            height = max(p.get_height(), patches_p[i].get_height())
            x, y = p.get_xy()
            ax.annotate(f'{saving}%\n{saving_m3}m3', (x + width/2, y + height*1.01), ha='center')
    #
    plt.ylabel('Monthly gas use [m3]')
    plt.title(building)
    plt.show(block=False)

In [143]:
# calculate monthly portfolio savings
gas_use_pred_sav = gas_use_pred
gas_use_meas_sav = gas_use_meas
for building in gas_eans.index:
    gas_use_pred_sav.loc[gas_use_pred_sav.index[0]:socs[building], building] = np.nan
    gas_use_meas_sav.loc[gas_use_meas_sav.index[0]:socs[building], building] = np.nan
gas_use_sav = gas_use_pred_sav - gas_use_meas_sav
gas_use_sav = gas_use_sav.resample('m').sum()
gas_use_sav = gas_use_sav.loc[socs[gas_eans.index].min():]

In [149]:
round(gas_use_sav.sum().sum(),2)

215564.23

In [ ]:
for building in gas_use_sav.columns:
    print(building, "total savings since SOC (m3):", gas_use_sav[building].sum())

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(25, 5))
for col, ax in zip(sorted(total_savings_plot_df.columns), axs.ravel()):
    total_savings_plot_df[col].plot.bar(ax=ax, color=['gold'])
    ax.set_xticklabels([x for x in total_savings_plot_df.index], rotation=45)
    ax.set_xlabel('Building')
    ax.set_ylabel(col)
plt.show(block=False)

In [ ]:
for building in gas_eans.index:
    fig, ax = plt.subplots(figsize=(25, 8))
    gas_use_train[building].plot(ax=ax, color='gray')
    gas_use_meas[building].plot(ax=ax, color='gold')
    ax.set_xlim(left=gas_use_train[building].dropna().index[0])
    ax.set_title(building)
    ax.set_ylabel('Gas use (m3)')
    ax.axvline(x=socs[building], color='k', ls='--')
    plt.show(block=False)

### Monthly

In [157]:
gas_use_pred_sav = gas_use_pred
gas_use_meas_sav = gas_use_meas
for building in gas_eans.index:
    gas_use_pred_sav.loc[gas_use_pred_sav.index[0]:socs[building], building] = np.nan
    gas_use_meas_sav.loc[gas_use_meas_sav.index[0]:socs[building], building] = np.nan
gas_use_sav = gas_use_pred_sav - gas_use_meas_sav
gas_use_sav = gas_use_sav.resample('m').sum()
gas_use_sav = gas_use_sav.loc[socs[gas_eans.index].min():]
gas_use_sav['m3_cost']=0.87 #assuming cost of m3 is 0.76 cents, could be changed
for col in gas_use_sav.columns:
    gas_use_sav[col]=round(gas_use_sav['m3_cost']*gas_use_sav[col],2)
del gas_use_sav['m3_cost']

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(20, 10))
plt.subplots_adjust(hspace=0.3)
for col, ax in zip(sorted(gas_use_sav.columns), axs.ravel()):
    index=gas_use_sav[col].index.get_loc(gas_use_sav[col].ne(0).idxmax())
    gas_use_sav[[col]].iloc[index:-1].plot(ax=ax, color='gold', linewidth=3) #excludes April for now, too early in the month to draw results
    ax.set_title(col.upper())
    ax.get_legend().remove()
    ax.set_ylabel("Saving EUR")

# Forecasting

In [ ]:
#forecast periods are defined, which is - per building - SOC up until 31st of March 2022
forecast_periods=pd.DataFrame()
forecast_periods['start'] = socs[gas_eans.index]
forecast_periods['end'] = pd.Timestamp('2022-03-31')
forecast_periods=forecast_periods[~forecast_periods.index.duplicated(keep='first')]

In [102]:
engine = Engine(project=project, isAsync=False)
time_res = 'd'
gas_use_train = pd.DataFrame(index=pd.date_range(start=forecast_periods['start'].min(),
                                                 end=forecast_periods['end'].max(),
                                                 freq=time_res))

# data from sp-ql
for building in gas_eans.index:
    # get gas use
    start_train = forecast_periods.loc[building, 'start']
    end_train = forecast_periods.loc[building, 'end']
    gas_ean = gas_eans[building]
    gas_use = engine.select(gas_ean).group_by(time_res).sum().after(start_train).before(end_train).all()
    gas_use = gas_use.rename(columns={gas_use.columns[0]:building})
    gas_use_train = gas_use_train.join(gas_use)
gas_use_train.to_csv(r'c2_gas/data/gas_use_train_f.csv')

In [103]:
gas_use_train = pd.read_csv(r'c2_gas/data/gas_use_train_f.csv', index_col=0,
                            parse_dates=True)
# some buildings have small gaps in gas data. We can interpolate over these
for building in gas_eans.index:
    start_train = forecast_periods.loc[building, 'start']
    end_train = forecast_periods.loc[building, 'end']
    gas_use_train_build = gas_use_train.loc[start_train:end_train, building]
    # setting first element to 0 if nan. To aid interpolation.
    if gas_use_train_build.iloc[[0]].isna()[0]:
        gas_use_train_build.iloc[0] = 0
    gas_use_train_build = gas_use_train_build.interpolate(limit=10) # filling max 10 consecutive days of missing data
    gas_use_train.loc[start_train:end_train, building] = gas_use_train_build

### Random Forest

In [ ]:
for building in gas_eans.index:
    print(fr'Training building {building}, {gas_eans.index.get_loc(building) + 1}/{len(gas_eans)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = forecast_periods.loc[building, 'start']
    end_train = forecast_periods.loc[building, 'end']

    if knmi_station==240:
        features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
    if knmi_station==344:
        features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
    if knmi_station==370:
        features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
    if knmi_station==260:
        features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
    if knmi_station==275:
        features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)
        
    del features['SQ'] #is only used for electricity
    
    features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')
    features['dow'] = features.index.dayofweek
    gas_use_train_build = gas_use_train[building]
    
    # X: training features, y: training target
    X = features[(features.index >= start_train) & (features.index < end_train)]
    y = gas_use_train_build[(gas_use_train_build.index >= start_train) &
                            (gas_use_train_build.index < end_train)]
    #
    # checking if there is any nan in the data
    if X.isna().sum().sum() + y.isna().sum() > 0:
        raise ValueError('NaN in training data:', building)
    #
    # checking on dependency of taget on features to get a feel
    if False:
        fig, ax = plt.subplots(len(X.columns), 1)
        for i, c in enumerate(X.columns):
            ax[i].scatter(x=X[c], y=y)
            ax[i].set_title(c)
        plt.show(block=False)
    #
    # make train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    #
    # grid-search for hyper parameter tuning
    param_grid = {'max_depth': [2, 5, 10, 20, 50, 100],
                  'min_samples_leaf': [1, 5, 10],
                  'n_estimators': [500]}
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                               cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)
    #
    # save the trained RF
    joblib.dump(grid_search.best_estimator_, fr'c2_gas/data/trained_models/f_rf/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_gas/data/trained_models/f_rf/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_gas/data/trained_models/f_rf/y_test_{building}.joblib')

### Gradient Boosting

In [ ]:
warnings.filterwarnings("ignore")

for building in gas_eans.index:
    print(fr'Training building {building}, {gas_eans.index.get_loc(building) + 1}/{len(gas_eans.index)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = forecast_periods.loc[building, 'start']
    end_train = forecast_periods.loc[building, 'end']

    if knmi_station==240:
        features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
    if knmi_station==344:
        features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
    if knmi_station==370:
        features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
    if knmi_station==260:
        features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
    if knmi_station==275:
        features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)
        
    del features['SQ'] #is only used for electricity
    
    features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')
    features['dow'] = features.index.dayofweek
    gas_use_train_build = gas_use_train[building]
    
    X = features[(features.index >= start_train) & (features.index < end_train)]
    y = gas_use_train_build[(gas_use_train_build.index >= start_train) &
                            (gas_use_train_build.index < end_train)]


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    param_grid = {
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10],
    }

    rf = GradientBoostingRegressor()

    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                        cv=KFold(10, shuffle=True), n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)

    joblib.dump(grid_search.best_estimator_, fr'c2_gas/data/trained_models/f_gb/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_gas/data/trained_models/f_gb/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_gas/data/trained_models/f_gb/y_test_{building}.joblib')

### XGBoosting

In [ ]:
for building in gas_eans.index:
    print(fr'Training building {building}, {gas_eans.index.get_loc(building) + 1}/{len(gas_eans)}')
    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]
    start_train = forecast_periods.loc[building, 'start']
    end_train = forecast_periods.loc[building, 'end']

    if knmi_station==240:
        features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
    if knmi_station==344:
        features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
    if knmi_station==370:
        features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
    if knmi_station==260:
        features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
    if knmi_station==275:
        features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)
        
    del features['SQ'] #is only used for electricity
    
    features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')
    features['dow'] = features.index.dayofweek
    gas_use_train_build = gas_use_train[building]
    
    X = features[(features.index >= start_train) & (features.index < end_train)]
    y = gas_use_train_build[(gas_use_train_build.index >= start_train) &
                            (gas_use_train_build.index < end_train)]
    if X.isna().mean().mean() + y.isna().mean() > 0:
        raise ValueError('NaN in training data:', building)
    if False:
        fig, ax = plt.subplots(len(X.columns), 1)
        for i, c in enumerate(X.columns):
            ax[i].scatter(x=X[c], y=y)
            ax[i].set_title(c)
        plt.show(block=False)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    
    param_grid={
                "learning_rate": (0.05, 0.10, 0.15, 0.2),
                "max_depth": [3, 4, 5, 6, 8],
                "min_child_weight": [1, 3, 5, 7]
                }

    rf = XGBRegressor()
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                               cv=cv, n_jobs=-1, verbose=False)
    grid_search.fit(X_train, y_train)

    joblib.dump(grid_search.best_estimator_, fr'c2_gas/data/trained_models/f_xg/trained_{building}.joblib')
    joblib.dump(X_test, fr'c2_gas/data/trained_models/f_xg/X_test_{building}.joblib')
    joblib.dump(y_test, fr'c2_gas/data/trained_models/f_xg/y_test_{building}.joblib')

### Predictions

In [111]:
gas_use_pred = pd.DataFrame(index=features.index)
for m in ['rf','gb','xg']:
    for building in gas_eans.index:
        
        city = cities[building]
        knmi_station = knmi_station_lookup[city]
        knmi_weather = weather[knmi_station]
        
        if knmi_station==240:
            features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
        if knmi_station==344:
            features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
        if knmi_station==370:
            features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
        if knmi_station==260:
            features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
        if knmi_station==275:
            features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)

        features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')
        del features['SQ'] #is only used for electricity
        features['dow'] = features.index.dayofweek
        #features['month'] = features.index.month
        X_pred = features
        # do the predictions
        models = os.listdir(f'c2_gas/data/trained_models/f_{m}')
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/f_{m}/trained_{building}.joblib')
        y_pred = grid_search.predict(X_pred)
        gas_use_pred[building+'_'+m] = y_pred

In [112]:
#ensemble stacking, equal weight
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3
gas_use_pred['*****_en']=(gas_use_pred['*****_rf']+gas_use_pred['*****_gb']+gas_use_pred['*****_xg'])/3

### Actual

In [106]:
# compare predicted and measured gas use after start of control
engine = Engine(project=project, isAsync=False)
time_res = 'd'
gas_use_meas = pd.DataFrame(index=pd.date_range(start=forecast_periods['start'].min(),
                                                end=knmi_weather.index[-49]+dt.timedelta(30),
                                                 freq=time_res))
for building in gas_eans.index:
    # get gas use
    start = train_periods['start'].min()
    end = knmi_weather.index[-1]+dt.timedelta(28)
    gas_ean = gas_eans[building]
    gas_use = engine.select(gas_ean).group_by(time_res).sum().after(start).before(end).all()
    gas_use = gas_use.rename(columns={gas_use.columns[0]:building})
    gas_use_meas = gas_use_meas.join(gas_use)

In [107]:
def calc_date_pos_on_plot(df_compare_monthly):
    soc=pd.Timestamp('2022-04-01 00:00:00')
    sm = list(df_compare_monthly.index < soc).index(False) # first False is start month
    return sm - 0.5 # place of dashed line on plot in 0-1 x coord range

In [ ]:
for building in gas_eans.index:
    start = forecast_periods.loc[building, 'start']
    end = gas_use_pred.index[-1]
    df_compare = pd.DataFrame(index=pd.date_range(start=start,
                                                     end=end,
                                                     freq=time_res))
    df_compare = df_compare.join(gas_use_meas[[building]])
    df_compare = df_compare.rename(columns={building:fr'{building}_meas'})
    df_compare = df_compare.join(gas_use_pred[[building+"_rf"]])
    #
    df_compare = df_compare.resample('m').sum()

    soc = socs[building]#.date()
    soc_pos = calc_date_pos_on_plot(df_compare_monthly=df_compare)
    train_end_pos = calc_date_pos_on_plot(df_compare_monthly=df_compare)
    #
    fig, ax = plt.subplots(figsize=(25, 8))
    df_compare.plot.bar(ax=ax, color=['gold', 'darkgrey'])
    #soc_mark = pd.Series(index=df_compare.index, data=np.nan)
    ax.set_xticklabels([x.strftime("%Y-%m") for x in df_compare.index], rotation=45)
    plt.vlines(x=train_end_pos, ymin=0, ymax=df_compare.max().max(), color='darkred', ls='--')
    plt.vlines(x=soc_pos, ymin=0, ymax=df_compare.max().max(), color='k', ls='--')
    patches = ax.patches
    patches_m = patches[0:int(len(patches)/2)]
    patches_p = patches[int(len(patches)/2):]

    plt.ylabel('Monthly electricity use [m3]')
    plt.title(building)
    plt.show(block=False)

In [119]:
metric_com_mae = pd.DataFrame(index=gas_eans.index)
regressors = ['rf', 'gb', 'xg']
for regressor in regressors:
    mae_range_percent = pd.Series()
    for building in gas_eans.index:
        models = os.listdir(f'c2_gas/data/trained_models/f_{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
        #
        # grid search already contains the re-trained model with the best parameters
        y_test_pred = grid_search.predict(X_test)

        #
        # calculating MAE and plotting test and pred_test
        mae = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
        # mae in percent of range
        mae_range_percent[building] = mae / y_test.max() * 100

        gas_use_train_build = gas_eans[building]

    metric_com_mae[regressor.upper()]=mae_range_percent

In [120]:
metric_com_rmse = pd.DataFrame(index=gas_eans.index)
regressors = ['rf', 'gb', 'xg']
for regressor in regressors:
    rmse_range_percent = pd.Series()
    for building in gas_eans.index:
        models = os.listdir(f'c2_gas/data/trained_models/f_{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
        #
        # grid search already contains the re-trained model with the best parameters
        y_test_pred = grid_search.predict(X_test)
        
        #calculating RMSE
        MSE = np.square(np.subtract(y_test, y_test_pred)).mean() 
        RMSE = math.sqrt(MSE)
        rmse_range_percent[building]=RMSE / y_test.max() * 100

    metric_com_rmse[regressor.upper()]=rmse_range_percent

In [121]:
en_mae = pd.DataFrame(index=gas_eans.index)
regressors = ['rf', 'gb', 'xg']
mae_range_percent = pd.Series()
master_test=pd.DataFrame()
master_test_pred=pd.DataFrame()

df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df_c=pd.DataFrame()

df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df_d=pd.DataFrame()

for building in gas_eans.index:
    for regressor in regressors:
        models = os.listdir(f'c2_gas/data/trained_models/f_{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
            #print(y_test) ##gives diffent dates, so needs to be investigated --> tried setting random seed during training
            y_test_pred = grid_search.predict(X_test)
            
            if regressor=="rf":
                df1=y_test.to_frame()
                df4=pd.DataFrame(y_test_pred, columns=[building])
                df1.reset_index(drop=True, inplace=True)
                df4.reset_index(drop=True, inplace=True)
            if regressor=="gb":
                df2=y_test.to_frame()
                df5=pd.DataFrame(y_test_pred, columns=[building])
                df2.reset_index(drop=True, inplace=True)
                df5.reset_index(drop=True, inplace=True)
            if regressor=="xg":
                df3=y_test.to_frame()
                df6=pd.DataFrame(y_test_pred, columns=[building])
                df3.reset_index(drop=True, inplace=True)
                df6.reset_index(drop=True, inplace=True)
                
    df_c=pd.concat([df1, df2, df3], axis=1)
    df_c.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_c[building+"_en"]=(df_c[building+"_rf"]+df_c[building+"_gb"]+df_c[building+"_xg"])/3
    del df_c[building+"_rf"]
    del df_c[building+"_gb"]
    del df_c[building+"_xg"]
    
    df_d=pd.concat([df4, df5, df6], axis=1)
    df_d.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_d[building+"_en"]=(df_d[building+"_rf"]+df_d[building+"_gb"]+df_d[building+"_xg"])/3
    
    del df_d[building+"_rf"]
    del df_d[building+"_gb"]
    del df_d[building+"_xg"]

    mae = mean_absolute_error(y_true=df_c[building+"_en"], y_pred=df_d[building+"_en"])
    
    mae_range_percent[building] = mae / df_c[building+"_en"].max() * 100

    en_mae['EN']=mae_range_percent
    

### MAEs

In [ ]:
metric_com_mae['EN']=en_mae
print(metric_com_mae)
print("\n")
print("Average MAEs")
round(metric_com_mae.mean(), 5)

In [123]:
en_rmse = pd.DataFrame(index=gas_eans.index)
regressors = ['rf', 'gb', 'xg']
rmse_range_percent = pd.Series()
master_test=pd.DataFrame()
master_test_pred=pd.DataFrame()

df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df_c=pd.DataFrame()

df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df_d=pd.DataFrame()

for building in gas_eans.index:
    for regressor in regressors:
        models = os.listdir(f'c2_gas/data/trained_models/f_{regressor}') #change for RF, GB, XG
        if fr'trained_{building}.joblib' in models:
            grid_search = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/trained_{building}.joblib') #change for RF, GB, XG
            X_test = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/X_test_{building}.joblib') #change for RF, GB, XG
            y_test = joblib.load(fr'c2_gas/data/trained_models/f_{regressor}/y_test_{building}.joblib') #change for RF, GB, XG
            #print(y_test) ##gives diffent dates, so needs to be investigated --> tried setting random seed during training
            y_test_pred = grid_search.predict(X_test)
            
            if regressor=="rf":
                df1=y_test.to_frame()
                df4=pd.DataFrame(y_test_pred, columns=[building])
                df1.reset_index(drop=True, inplace=True)
                df4.reset_index(drop=True, inplace=True)
            if regressor=="gb":
                df2=y_test.to_frame()
                df5=pd.DataFrame(y_test_pred, columns=[building])
                df2.reset_index(drop=True, inplace=True)
                df5.reset_index(drop=True, inplace=True)
            if regressor=="xg":
                df3=y_test.to_frame()
                df6=pd.DataFrame(y_test_pred, columns=[building])
                df3.reset_index(drop=True, inplace=True)
                df6.reset_index(drop=True, inplace=True)
                
    df_c=pd.concat([df1, df2, df3], axis=1)
    df_c.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_c[building+"_en"]=(df_c[building+"_rf"]+df_c[building+"_gb"]+df_c[building+"_xg"])/3
    del df_c[building+"_rf"]
    del df_c[building+"_gb"]
    del df_c[building+"_xg"]
    
    df_d=pd.concat([df4, df5, df6], axis=1)
    df_d.columns = [fr'{building}_rf', fr'{building}_gb',fr'{building}_xg']
    df_d[building+"_en"]=(df_d[building+"_rf"]+df_d[building+"_gb"]+df_d[building+"_xg"])/3
    

    MSE = np.square(np.subtract(df_c[building+"_en"], df_d[building+"_en"])).mean() 
    RMSE = math.sqrt(MSE)
    rmse_range_percent[building]=RMSE / df_c[building+"_en"].max() * 100
    
    en_rmse['EN']=rmse_range_percent
    

In [ ]:
metric_com_rmse['EN']=en_rmse
print(metric_com_rmse)
print("\n")
print("Average RMSEs")
round(metric_com_rmse.mean(), 5)

### Individual Predictions

In [159]:
gas_use_pred = pd.DataFrame(index=features.index)
for building in gas_eans.index:

    city = cities[building]
    knmi_station = knmi_station_lookup[city]
    knmi_weather = weather[knmi_station]

    if knmi_station==240:
        features = pd.read_csv('features_240.csv', index_col=0, squeeze=True)
    if knmi_station==344:
        features = pd.read_csv('features_344.csv', index_col=0, squeeze=True)
    if knmi_station==370:
        features = pd.read_csv('features_370.csv', index_col=0, squeeze=True)
    if knmi_station==260:
        features = pd.read_csv('features_260.csv', index_col=0, squeeze=True)
    if knmi_station==275:
        features = pd.read_csv('features_275.csv', index_col=0, squeeze=True)

    features.index = pd.date_range('2011-01-01', periods=4473, freq='1D')
    del features['SQ'] #is only used for electricity
    features['dow'] = features.index.dayofweek
    #features['month'] = features.index.month
    X_pred = features
    # do the predictions
    models = os.listdir(f'c2_gas/data/trained_models/f_xg')
    if fr'trained_{building}.joblib' in models:
        grid_search = joblib.load(fr'c2_gas/data/trained_models/f_xg/trained_{building}.joblib')
    y_pred = grid_search.predict(X_pred)
    gas_use_pred[building] = y_pred

In [ ]:
gas_use_pred=gas_use_pred.loc['2022-04-01':,]
gas_use_pred=gas_use_pred.resample('m').sum()
gas_use_pred.mean()*0.87